<a href="https://colab.research.google.com/github/sakkarin31/musicrecomment/blob/main/miniMLmusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# โหลด Dataset
df = pd.read_csv("dataset.csv")

# เลือกเฉพาะ Feature ที่ใช้เทรนโมเดล
features = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo']
X = df[features].values

# Normalize ข้อมูล
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# กำหนดขนาด Input
input_dim = X_scaled.shape[1]

# Encoder (เพิ่มจำนวนชั้นและจำนวนหน่วยในแต่ละชั้น)
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)  # เพิ่มขนาดเป็น 64
encoded = Dense(32, activation='relu')(encoded)  # เพิ่มขนาดเป็น 32
encoded = Dense(16, activation='relu')(encoded)  # ขนาด 16

# Decoder (เปลี่ยน activation เป็น linear เพื่อให้สามารถสร้างค่าที่ไม่ถูกจำกัด)
decoded = Dense(32, activation='relu')(encoded)  # ขนาด 32
decoded = Dense(64, activation='relu')(decoded)  # ขนาด 64
decoded = Dense(input_dim, activation='linear')(decoded)  # ใช้ linear เพราะค่าไม่ควรถูกจำกัด

# สร้างโมเดล Autoencoder
autoencoder = Model(input_layer, decoded)

# ใช้ Adam optimizer และ Mean Squared Error loss function
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')


In [6]:
# เทรนโมเดล
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=32, shuffle=True)


Epoch 1/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 0.1068
Epoch 2/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 7.9401e-04
Epoch 3/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 2.7048e-04
Epoch 4/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 2.6774e-04
Epoch 5/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 2.5745e-04
Epoch 6/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 1.8057e-04
Epoch 7/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 1.8548e-04
Epoch 8/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 2.2367e-04
Epoch 9/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 1.7147e-04
Epoch 10/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 2.4426e-04
Epoch 11/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 1.7017e-04
Epoch 12/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 2.6380e-04
Epoch 13/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 1.9393e-04
Epoch 14/50
3563/3563 ━━━━━━━━━━━━━━━━━━━━ 10

In [32]:
# สร้าง Encoder Model
encoder = Model(input_layer, encoded)

# แปลงเพลงเป็นเวกเตอร์
X_encoded = encoder.predict(X_scaled)

from sklearn.metrics.pairwise import cosine_similarity

# หาเพลงที่คล้ายกัน โดยใช้ชื่อเพลงและชื่อนักร้อง
def recommend_similar(song_name, artist_name, df, top_n=5):
    # ค้นหาเพลงและนักร้องที่ตรงกันใน DataFrame
    song_index = df[(df['track_name'].str.lower() == song_name.lower()) &
                    (df['artists'].str.lower() == artist_name.lower())].index

    if len(song_index) == 0:
        return "❌ ไม่พบเพลงนี้ใน Dataset"

    # ดึงเวกเตอร์ของเพลงที่ค้นหา
    song_vector = X_encoded[song_index]

    # คำนวณความคล้ายคลึงระหว่างเพลงที่เลือกและเพลงใน dataset
    similarity = cosine_similarity(song_vector, X_encoded)

    # หาเพลงที่คล้ายที่สุด (top_n เพลง) โดยใช้การจัดอันดับ
    indices = similarity.argsort()[0][-top_n-1:-1][::-1]
    recommended_songs = df.iloc[indices]

    # กรองเพลงที่เหมือนกันทั้งชื่อเพลงและศิลปินออกจากผลลัพธ์
    recommended_songs = recommended_songs[~((recommended_songs['track_name'].str.lower() == song_name.lower()) &
                                            (recommended_songs['artists'].str.lower() == artist_name.lower()))]
    #กรองซ้ำจากเพลงที่แนะนำแล้ว
    recommended_songs = recommended_songs.drop_duplicates(subset=['track_name', 'artists'])
    # ตรวจสอบว่าจำนวนเพลงที่แนะนำยังไม่ครบตามจำนวนที่ต้องการ
    num_recommended = len(recommended_songs)

    # เก็บชื่อเพลงและนักร้องที่แนะนำแล้ว
    recommended_song_titles = set(recommended_songs['track_name'].str.lower() + " " + recommended_songs['artists'].str.lower())

    # กรณีที่แนะนำเพลงยังไม่ครบจำนวน top_n
    if num_recommended < top_n:
        # หาเพลงที่คล้ายกันเพิ่มเติมจากที่เหลือ
        remaining_indices = similarity.argsort()[0][::-1]  # เลือกเพลงที่คล้ายที่สุดทั้งหมด
        remaining_songs = df.iloc[remaining_indices]

        # กรองเพลงที่ซ้ำ (ชื่อเพลงและนักร้องเดียวกัน) ออก
        remaining_songs = remaining_songs[~((remaining_songs['track_name'].str.lower() == song_name.lower()) &
                                            (remaining_songs['artists'].str.lower() == artist_name.lower()))]

        # ลบเพลงที่เคยแนะนำไปแล้วจาก remaining_songs
        remaining_songs = remaining_songs[~((remaining_songs['track_name'].str.lower() + " " + remaining_songs['artists'].str.lower()).isin(recommended_song_titles))]

        # เอาผลลัพธ์ที่เหลือมารวมกันเพื่อให้ได้จำนวนที่ต้องการ
        recommended_songs = pd.concat([recommended_songs, remaining_songs]).drop_duplicates(subset=['track_name', 'artists'])

    # ตรวจสอบให้ได้จำนวนเพลงที่แนะนำครบตาม top_n
    recommended_songs = recommended_songs.head(top_n)

    # หากไม่พบเพลงที่คล้ายคลึงกัน (กรณีที่กรองแล้วไม่มีเพลงในรายการแนะนำ)
    if recommended_songs.empty:
        return "❌ ไม่มีเพลงที่คล้ายคลึงกันใน Dataset"

    return recommended_songs[['track_name', 'artists']]

# ทดสอบแนะนำเพลง โดยใช้ชื่อเพลงและนักร้อง
print(recommend_similar("Starboy", "The Weeknd;Daft Punk", df))


3563/3563 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
               track_name                                 artists
9314         Sem a Planta                       ConeCrewDiretoria
111438       Baseball Cap  Faithless;Rollo Armstrong;Sister Bliss
60119   Verified Choppa 2                                Marksman
111953      Lonely & Cold                                Gramatik
60678            Nice Suh                             Vybz Kartel
